In [1]:
from torch.utils.data import DataLoader

from data.dataset_class import MultiLabelDataset
from base_model import ResNetModel
from run_train import run_epochs


d:\pr\cnn_with_resnet\.venv\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [2]:
base_model = ResNetModel()

In [3]:
model = base_model.get_model

In [4]:
train_data = DataLoader(MultiLabelDataset("train"), batch_size=64, shuffle=False)
val_data = DataLoader(MultiLabelDataset("val"), batch_size=64, shuffle=False)
test_data = DataLoader(MultiLabelDataset("test"), batch_size=64, shuffle=False)

In [ ]:
history = run_epochs(1, train_data, val_data, model)

 55%|█████▌    | 53/96 [01:56<01:34,  2.20s/it]                                    

In [13]:
example_data = MultiLabelDataset("test")

img, target = example_data[123]
model.eval()
pred = model(img.unsqueeze(0))

print("Target: ", target)
print("Predict", pred)

Target:  tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.])
Predict tensor([[ -16.2434,  -18.0229,  -80.6294,   -1.2821,   -0.3796, -109.2982,
          -11.6036,   -6.1859,   -2.1220,   -3.2324,  -49.6537, -177.2017,
            2.1375,  -62.1905,  -28.5349,  -15.5149]],
       grad_fn=<AddmmBackward0>)
